In [1]:
import json
import tqdm 
import os
import sys 

In [4]:
train_path = "/home2/tathagato/summarization/MACSUM/dataset/macdoc/train.json"
val_path = "/home2/tathagato/summarization/MACSUM/dataset/macdoc/val.json"
test_path = "/home2/tathagato/summarization/MACSUM/dataset/macdoc/test.json"


In [16]:
#first understand the structure of the data
test_data = json.load(open(test_path))
print(test_data[0].keys())
for key in test_data[0]['references']:
    print(key)
print(test_data[0]['references'])
print(test_data[0]['source'])

dict_keys(['metadata', 'references', 'source'])
{'control_attribute': {'length': 'short', 'extractiveness': 'normal', 'specificity': 'normal', 'topic': ''}, 'summary': 'The 2014 Khumbu Icefall on Everest caused 16 Nepali deaths last year, resulting in at the end of the Nepal season. Mountaineers return to Everest after the deadly 2014 season. To avoid the deadly area, changes were made to the routes. Climbing permits have increased this year.', 'title': 'Nepal Opens Mount Everest Side After 2014 Tragedy', 'relevant_turns': [[0, 49]]}
{'control_attribute': {'length': 'normal', 'extractiveness': 'normal', 'specificity': 'normal', 'topic': ''}, 'summary': "In 2014, 16 Nepali died after glacial ice caused an avalanche, ending the Nepal Mount Everest climbing season. This year, mountaineers are returning to Everest and will be taking an alternate route to avoid the treacherous and dangerous path. There's been a slight increase in the number of climbing permits issued this year in Nepal, as 

In [30]:
def generate_dpo_pairs(data, attributes):
    dataset = {}
    new_idx = 0
    for idx, example in tqdm.tqdm(enumerate(data)):
        source = example['source']
        num_references = len(example['references'])
        for i in range(num_references):
            for j in range(i+1, num_references):
                first_control_attributes = example['references'][i]['control_attribute']
                second_control_attributes = example['references'][j]['control_attribute']
                is_usable = True
                for attr in attributes:
                    if first_control_attributes[attr] == "" or second_control_attributes[attr] == "":
                        is_usable = False
                        break
                    elif first_control_attributes[attr] == second_control_attributes[attr]:
                        is_usable = False
                        break
                if is_usable:
                    first_selection = {attr : first_control_attributes[attr] for attr in attributes}
                    second_selection = {attr : second_control_attributes[attr] for attr in attributes}
                    dataset[new_idx] = { "source" : source , "chosen" : example['references'][i]['summary'], "unchosen" : example['references'][j]['summary'], "prefered_control_attributes" : first_selection, "rejected_control_attributes" : second_selection, "indexes" : [i, j]}
                    new_idx += 1
                    dataset[new_idx] = { "source" : source , "chosen" : example['references'][j]['summary'], "unchosen" : example['references'][i]['summary'], "prefered_control_attributes" : second_selection, "rejected_control_attributes" : first_selection, "indexes" : [j, i]}
                    new_idx += 1
    return dataset
train_dataset = generate_dpo_pairs(test_data, ["length"])



94it [00:00, 13169.37it/s]


In [31]:
len(train_dataset)

1938

In [32]:
for idx, key in enumerate(train_dataset.keys()):
    print(train_dataset[key]['prefered_control_attributes'], train_dataset[key]['rejected_control_attributes'], train_dataset[key]['indexes'])
    if idx == 27:
        break

{'length': 'short'} {'length': 'normal'} [0, 1]
{'length': 'normal'} {'length': 'short'} [1, 0]
{'length': 'short'} {'length': 'long'} [0, 2]
{'length': 'long'} {'length': 'short'} [2, 0]
{'length': 'short'} {'length': 'normal'} [0, 3]
{'length': 'normal'} {'length': 'short'} [3, 0]
{'length': 'short'} {'length': 'normal'} [0, 4]
{'length': 'normal'} {'length': 'short'} [4, 0]
{'length': 'short'} {'length': 'normal'} [0, 5]
{'length': 'normal'} {'length': 'short'} [5, 0]
{'length': 'normal'} {'length': 'long'} [1, 2]
{'length': 'long'} {'length': 'normal'} [2, 1]
{'length': 'normal'} {'length': 'short'} [1, 6]
{'length': 'short'} {'length': 'normal'} [6, 1]
{'length': 'long'} {'length': 'normal'} [2, 3]
{'length': 'normal'} {'length': 'long'} [3, 2]
{'length': 'long'} {'length': 'normal'} [2, 4]
{'length': 'normal'} {'length': 'long'} [4, 2]
{'length': 'long'} {'length': 'normal'} [2, 5]
{'length': 'normal'} {'length': 'long'} [5, 2]
{'length': 'long'} {'length': 'short'} [2, 6]
{'leng